In [1]:
import numpy as np

In [2]:
########################################
# Target instrument to be calibrated   #
########################################

gl_R_X = (3197, 6)
gl_R_Y = (3094, 21)
gl_R_Z = (2777, 4999)
gl_G_X = (3062, 29)
gl_G_Y = (2965, 5)
gl_G_Z = (3057, 257)
gl_B_X = (242.7, 1176)
gl_B_Y = (1676, 29)
gl_B_Z = (2145, 20)
gl_W_X = (3212, 5)
gl_W_Y = (3273, 4)
gl_W_Z = (3183, 18)

########################################
# Reference instrument measurement     #
########################################

# Tristimulus measurement from a reference instrument
ref_R_X = 3287
ref_G_X = 1141
ref_B_X = 561
ref_W_X = 4989
ref_R_Y = 1417
ref_G_Y = 3539
ref_B_Y = 325
ref_W_Y = 5282
ref_R_Z = 1.88
ref_G_Z = 108
ref_B_Z = 4271
ref_W_Z = 4381

ref_R_Cx = 0.6908
ref_G_Cx = 0.2398
ref_B_Cx = 0.1247
ref_W_Cx = 0.3665
ref_R_Cy = 0.3077
ref_G_Cy = 0.7224
ref_B_Cy = 0.0798
ref_W_Cy = 0.3874
ref_R_Lv = 1614
ref_G_Lv = 3757
ref_B_Lv = 343.5
ref_W_Lv = 5675

# ins_raw is the raw X/Y/Z tristimulus measurement (Graylevel, or GL, 12-bit) from the target imaging colorimeter instrument
ins_raw = [[gl_R_X, gl_G_X, gl_B_X, gl_W_X],
           [gl_R_Y, gl_G_Y, gl_B_Y, gl_W_Y],
           [gl_R_Z, gl_G_Z, gl_B_Z, gl_W_Z],
          ]
ins_raw = np.array(ins_raw)
# print('raw data for from instrument: (Graylevel, exposure_time_in_ms)\n', ins_raw)

# TODO: dark field correction must be performed for ins_raw based on exposure time and tempaerature

def normalize_gl(m_raw):
    m_raw = np.array(m_raw)
    m_norm = m_raw[:, :, 0] / m_raw[:, :, 1]
    return m_norm

ins_tristimulus_norm = normalize_gl(ins_raw)

print('Normalized instrument tristimulus rgbw matrix: Graylevel/ms\n', ins_tristimulus_norm)

ref_tristimulus_rgbw = [[ref_R_X, ref_G_X, ref_B_X, ref_W_X],
                        [ref_R_Y, ref_G_Y, ref_B_Y, ref_W_Y],
                        [ref_R_Z, ref_G_Z, ref_B_Z, ref_W_Z],
                       ]
ref_chromalumin_rgbw = [[ref_R_Cx, ref_G_Cx, ref_B_Cx, ref_W_Cx],
                        [ref_R_Cy, ref_G_Cy, ref_B_Cy, ref_W_Cy],
                        [ref_R_Lv, ref_G_Lv, ref_B_Lv, ref_W_Lv],
                       ]

ref_tristimulus_rgbw = np.array(ref_tristimulus_rgbw)
ref_chromalumin_rgbw = np.array(ref_chromalumin_rgbw)

print('raw data from tristimulus reference (for mode 0):\n', ref_tristimulus_rgbw)
print('raw data from chromaticity-luminance reference (for mode 1):\n', ref_chromalumin_rgbw)

Normalized instrument tristimulus rgbw matrix: Graylevel/ms
 [[5.32833333e+02 1.05586207e+02 2.06377551e-01 6.42400000e+02]
 [1.47333333e+02 5.93000000e+02 5.77931034e+01 8.18250000e+02]
 [5.55511102e-01 1.18949416e+01 1.07250000e+02 1.76833333e+02]]
raw data from tristimulus reference (for mode 0):
 [[3.287e+03 1.141e+03 5.610e+02 4.989e+03]
 [1.417e+03 3.539e+03 3.250e+02 5.282e+03]
 [1.880e+00 1.080e+02 4.271e+03 4.381e+03]]
raw data from chromaticity-luminance reference (for mode 1):
 [[6.908e-01 2.398e-01 1.247e-01 3.665e-01]
 [3.077e-01 7.224e-01 7.980e-02 3.874e-01]
 [1.614e+03 3.757e+03 3.435e+02 5.675e+03]]


In [3]:
def get_tristimulus_r(tristimulus_rgbw):
    return tristimulus_rgbw[:, 0]
def get_tristimulus_g(tristimulus_rgbw):
    return tristimulus_rgbw[:, 1]
def get_tristimulus_b(tristimulus_rgbw):
    return tristimulus_rgbw[:, 2]
    
def get_tristimulus_rgb(tristimulus_rgbw):
    return tristimulus_rgbw[:, 0:3]

def get_tristimulus_white(tristimulus_rgbw):
    return tristimulus_rgbw[:, 3]
    
def get_luminance_white(tristimulus_rgbw):
    return tristimulus_rgbw[1, 3]

def get_row_y_from_R(R):
    return R[1, :]

def get_chromaticity_rgb(chromalumin_rgbw):
    N_chromaticity_rgb = np.zeros(shape=(3,3))
    N_chromaticity_rgb[0:2, :] = chromalumin_rgbw[0:2, 0:3]
    N_chromaticity_rgb[2,:] = 1 - N_chromaticity_rgb[0,:] - N_chromaticity_rgb[1,:]
    return N_chromaticity_rgb

def get_chromaticity_white(chromalumin_rgbw):
    N_chromaticity_white = np.zeros(shape=(3,))
    N_chromaticity_white[0:2] = chromalumin_rgbw[0:2, 3]
    N_chromaticity_white[2] = 1 - N_chromaticity_white[0] - N_chromaticity_white[1]
    # print('N_chromaticity_white\n', N_chromaticity_white)
    return N_chromaticity_white

def get_luminance_white_from_chromalumin(chromalumin_rgbw):
    return chromalumin_rgbw[2, 3]
    
# mode 0: using tristimulus from reference
def mode0(ref_tristimulus_rgbw, ins_tristimulus_rgbw):
    N_tristimulus_rgb = get_tristimulus_rgb(ref_tristimulus_rgbw)
    M_tristimulus_rgb = get_tristimulus_rgb(ins_tristimulus_rgbw)
    R = np.matmul(N_tristimulus_rgb, np.linalg.inv(M_tristimulus_rgb))
    
    L_w = get_luminance_white(ref_tristimulus_rgbw)
    R_y = get_row_y_from_R(R)
    M_w = get_tristimulus_white(ins_tristimulus_rgbw)
    k_Y = L_w/np.matmul(R_y, M_w)
    
    R_prime = k_Y * R
    return R_prime

# mode 1: using Chromaticity and luminance from reference
def mode1(ref_chromalumin_rgbw, ins_tristimulus_rgbw):
    N_chromaticity_rgb = get_chromaticity_rgb(ref_chromalumin_rgbw)
    
    N_chromaticity_white = get_chromaticity_white(ref_chromalumin_rgbw)
    
    inv = np.linalg.inv(N_chromaticity_rgb)
    k_ref = np.matmul(inv, N_chromaticity_white)
    k_ref_diag = np.diag(k_ref)
    
    N__xyL_rgb = np.matmul(N_chromaticity_rgb, k_ref_diag)

    M_tristimulus_rgb = get_tristimulus_rgb(ins_tristimulus_rgbw)
    R = np.matmul(N__xyL_rgb, np.linalg.inv(M_tristimulus_rgb))

    L_w = get_luminance_white_from_chromalumin(ref_chromalumin_rgbw)
    R_y = get_row_y_from_R(R)
    M_w = get_tristimulus_white(ins_tristimulus_rgbw)
    k_Y = L_w/np.matmul(R_y, M_w)
    
    R_prime = k_Y * R
    return R_prime

In [4]:
R0 = mode0(ref_tristimulus_rgbw, ins_tristimulus_norm)
print('mode0 R\'\n', R0)

R1 = mode1(ref_chromalumin_rgbw, ins_tristimulus_norm)
print('mode1 R\'\n', R1)

mode0 R'
 [[ 5.82538977  0.75174822  4.7036043 ]
 [ 1.03853103  5.65825824 -0.08495086]
 [ 0.13805605 -0.63504854 39.32074092]]
mode1 R'
 [[ 6.37663676  0.82097172  4.41303799]
 [ 1.30394769  5.92971017 -0.08279743]
 [ 0.06925478 -0.3162911  31.22287328]]


In [5]:
# mode0 validation
print('mode0 validation')
M_measure = get_tristimulus_r(ins_tristimulus_norm)
M_corrected = np.matmul(R0, M_measure)
print(f'M_corrected: {M_corrected}')
M_measure = get_tristimulus_g(ins_tristimulus_norm)
M_corrected = np.matmul(R0, M_measure)
print(f'M_corrected: {M_corrected}')
M_measure = get_tristimulus_b(ins_tristimulus_norm)
M_corrected = np.matmul(R0, M_measure)
print(f'M_corrected: {M_corrected}')
M_measure = get_tristimulus_white(ins_tristimulus_norm)
M_corrected = np.matmul(R0, M_measure)
print(f'M_corrected: {M_corrected}')

def get_CxCyLv_from_XYZ(m):
    Cx = m[0]/np.sum(m)
    Cy = m[1]/np.sum(m)
    Lv = m[1]
    return Cx, Cy, Lv
print()

# mode1 validation
print('mode1 validation')
M_measure = get_tristimulus_r(ins_tristimulus_norm)
M_corrected = np.matmul(R1, M_measure)
Cx, Cy, Lv = get_CxCyLv_from_XYZ(M_corrected)
print(M_corrected)
print(f'Cx={Cx:0.4f}, Cy={Cy:0.4f}, Lv={Lv:0.2f}')

Cx = M_corrected[0]/np.sum(M_corrected)
M_measure = get_tristimulus_g(ins_tristimulus_norm)
M_corrected = np.matmul(R1, M_measure)
print(M_corrected)
Cx, Cy, Lv = get_CxCyLv_from_XYZ(M_corrected)
print(f'Cx={Cx:0.4f}, Cy={Cy:0.4f}, Lv={Lv:0.2f}')

M_measure = get_tristimulus_b(ins_tristimulus_norm)
M_corrected = np.matmul(R1, M_measure)
print(M_corrected)
Cx, Cy, Lv = get_CxCyLv_from_XYZ(M_corrected)
print(f'Cx={Cx:0.4f}, Cy={Cy:0.4f}, Lv={Lv:0.2f}')

M_measure = get_tristimulus_white(ins_tristimulus_norm)
M_corrected = np.matmul(R1, M_measure)
print(M_corrected)
Cx, Cy, Lv = get_CxCyLv_from_XYZ(M_corrected)
print(f'Cx={Cx:0.4f}, Cy={Cy:0.4f}, Lv={Lv:0.2f}')

mode0 validation
M_corrected: [3.21733232e+03 1.38696681e+03 1.84015356e+00]
M_corrected: [1116.81660514 3463.9912056   105.71094948]
M_corrected: [ 549.10965424  318.11165352 4180.47652985]
M_corrected: [5189.10239882 5282.         6522.27642452]

mode1 validation
[3521.09260953 1568.38476542    7.64568459]
Cx=0.6908, Cy=0.3077, Lv=1568.38
[1212.61394549 3653.01215272  191.14598474]
Cx=0.2398, Cy=0.7224, Lv=3653.01
[ 522.06082286  334.08543435 3330.38800788]
Cx=0.1247, Cy=0.0798, Lv=334.09
[5548.48377827 5675.         5306.92883755]
Cx=0.3357, Cy=0.3433, Lv=5675.00
